In [ ]:
import json

def dict2json(dict, jsonpath):
    try:
        with open(jsonpath, "w", encoding="utf-8") as f:
            json.dump(dict, f, ensure_ascii=False, indent=4)
    except Exception as er:
        print(f"⚠️ dict2json > Error: {er}")

def json2dict(jsonpath):
    dict = {}
    try:
        with open(jsonpath, "r", encoding="utf-8") as f:
            dict = json.load(f)
    except Exception as er:
        print(f"⚠️ json2dict > Error: {er}")
    return dict

In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import re

with open('Cổng Dịch vụ công Bộ Công an.html', 'r', encoding='utf-8') as file:
    html_content = file.read()
soup = BeautifulSoup(html_content, 'html.parser')
links = soup.select('ul.list-tthc a.tthc-name')
raw_list = []
for link in links:
    href = link.get('href')
    matt = None
    if 'matt=' in href:
        matt = href.split('matt=')[1].split('&')[0]
    name = link.get_text(strip=True)
    name = re.sub(r'\s+', ' ', name).strip()
    raw_list.append({'link': href, 'code': matt, 'name': name})
# raw_list

In [ ]:
DATA_data = []
for i in range(len(raw_list)):
    with open(f"html/{raw_list[i]['code']}.html", 'r', encoding='utf-8') as f:
        html_content = f.read()
    soup = BeautifulSoup(html_content, 'html.parser')
    items = soup.find_all(class_='tthc-list-item-detail')
    items = [item.contents for item in items]
    items = ["".join([str(e2) for e2 in e1]).strip() for e1 in items]
    items = [e.replace("<html>", "").replace("</html>", "") for e in items]
    items = [e.replace("<body>", "").replace("</body>", "") for e in items]
    items = [e.replace("\n", "") for e in items]
    items = [e.replace("\t", "") for e in items]
    items = [e.replace("\xa0", "") for e in items]
    items = [re.sub(r'<a\b[^>]*>.*?</a>', '', e, flags=re.DOTALL) for e in items]
    items = [re.sub(r'\s*(class|style)="[^"]*"', '', e) for e in items]
    items = [e.replace("<p></p>", "") for e in items]
    items = [e.replace("<b>", "").replace("</b>", "") for e in items]
    items = [e.replace("<i>", "").replace("</i>", "") for e in items]
    items = [e.replace("<strong>", "").replace("</strong>", "") for e in items]
    items = [e.strip() for e in items]

    fieldnames = [
        "Mã thủ tục",
        "Lĩnh vực",
        "Cơ quan thực hiện",
        "Mức độ cung cấp dịch vụ công trực tuyến",
        "Cách thức thực hiện",
        "Trình tự thực hiện",
        "Thời hạn giải quyết",
        "Phí",
        "Lệ Phí",
        "Thành phần hồ sơ",
        "Yêu cầu - điều kiện",
        "Căn cứ pháp lý",
        "Biểu mẫu",
        "Kết quả thực hiện"
    ]
    thutuc_object = {}
    thutuc_object["Tên thủ tục"] = raw_list[i]['name']
    for u in range(14):
        thutuc_object[fieldnames[u]] = items[u]
    thutuc_object_wrap = {
        "name": raw_list[i]['name'],
        "link": raw_list[i]['link'],
        "code": raw_list[i]['code'],
        "content": thutuc_object,
    }
    DATA_data.append(thutuc_object_wrap)

In [ ]:
THEDATA = json2dict("D:\Github\DVC_SearchAssistant_V3\static\DATA.json")
THEDATA["DVC_TTHC_BoCongAn"] = {
    "fieldnames": [
        "Tên thủ tục",
        "Mã thủ tục",
        "Lĩnh vực",
        "Cơ quan thực hiện",
        "Mức độ cung cấp dịch vụ công trực tuyến",
        "Cách thức thực hiện",
        "Trình tự thực hiện",
        "Thời hạn giải quyết",
        "Phí",
        "Lệ Phí",
        "Thành phần hồ sơ",
        "Yêu cầu - điều kiện",
        "Căn cứ pháp lý",
        "Biểu mẫu",
        "Kết quả thực hiện"
    ],
    "data": DATA_data
}
dict2json(THEDATA, "D:\Github\DVC_SearchAssistant_V3\static\DATA.json")